<a href="https://colab.research.google.com/github/Amanblackdevil/Machine-Learning/blob/main/Decision_tree_Classifier_IrisDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()
import io

Saving Iris.csv to Iris.csv


In [15]:
df = pd.read_csv(io.BytesIO(uploaded['Iris.csv']))

In [16]:
df.head(10)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


In [26]:
#remove unneed column
df.drop("Id", axis=1, inplace=True)

In [27]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [28]:
class Node():
  def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):

    #from decision tree node
    self.feature_index =  feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.info_gain = info_gain

    #for leaf node
    self.value = value

In [29]:
class DecisionTreeClassifier():
  def __init__(self, min_samples_split=2, max_depth=2):

    #initialize the root of the tree
    self.root = None

    #stopping condition
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth

    def build_tree(seld, dataset, curr_depth):

      X, Y = dataset[:, :-1], dataset[:, -1]
      num_samples, num_features = np.shape(X)

      #split until stopping condtion are met
      if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
        #find the best split
        best_split = self.get_best_split(dataset, num_samples, num_features)
        #check if information gain is positive
        if best_split['info_gain']>0:
          #recur left
          left_subtree = self.build_tree(best_split['dataset_left'], curr_depth+1)
          #ricur right
          right_subtree = self.build_tree(best_split['dataset_right'], curr_depth+1)

          #return the decision tree
          return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])


      #compute leaf node
      leaf_value = self.calculate_leaf_value(Y)
      #return leaf node
      return Node(value=leaf_value)

  def get_best_split(self, dataset, num_samples, num_features):
    #dictionary to store the best split
    best_split = {}
    max_info_gain = -float("inf")

    #loop over all the features
    for feature_index in range(num_features):
      feature_values = dataset[:, feature_index]
      possible_threshold = np.unique(feature_values)

      #loop over all the feature values present in data
      for threshold in possible_threshold:
        #get the current split
        dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
        #check if the child are not null
        if len(dataset_left)>0 and len(dataset_right)>0:
          y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]

          #compute information gain
          curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
          #update the best split if needed
          if curr_info_gain>max_info_gain:
            best_split["feature_index"] = feature_index
            best_split["threshold"] = threshold
            best_split["dataset_left"] = dataset_left
            best_split["dataset_right"] = dataset_right
            best_split["info_gain"] = curr_info_gain
            max_info_gain = curr_info_gain

    #return best split
    return best_split

  def split(self, dataset, feature_index, threshold):
    dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
    dataset_right = np.array([row for row in dataset if row[feature_index]<=threshold])
    return dataset_left, dataset_right

  def information_gain(self, parents, l_child, r_child, mode="entropy"):

    weight_l = len(l_child)/ len(parents)
    weight_r = len(r_child)/ len(parents)

    if mode=="gini":
      gain=self.gini_index(parents)-(weight_l*self.gini_index(l_child)+weight_r*self.gini_index(r_child))
    else:
      gain=self.gini_index(parents)-(weight_l*self.entropy(l_child)+weight_r*self.entropy(r_child))

    return gain

  def entropy(self, y):
    class_labels = np.unique(y)
    for cls in class_labels:
      p_cls = len(y[y == cls]) / len(y)
      entropy += -p_cls*np.log2(p_cls)

    return entropy

  def gini_index(self, y):
    class_labels = np.unique(y)
    gini=0
    for cls in class_labels:
      p_cls = len(y[y == cls])/len(y)
      gini += p_cls**2
    return 1-gini

  def calculate_leaf_value(self, Y):

    Y = list(Y)
    return max(Y, key=Y.count)

  def print_tress(self, tree=None, indent=" "):

    if not tree:
      tree = self.root

    if tree.value is not None:
      print(tree.value)

    else:
      print("X"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
      print("%sleft :" %(indent), end="")
      self.print_tree(tree.left, indent+indent)
      print("%sright :" %(indent), ent="")
      self.print_tree(tree.right, indent+indent)


  def fit(self, X, Y):
    dataset = np.concatenate((X, Y), axis=1)
    self.root = self.build_tree(dataset)

  def predict(self, X):
    prediction = [self.make_prediction(x, self.root) for x in X]
    return prediction


In [34]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [35]:
train.shape, test.shape

((120, 5), (30, 5))

In [43]:
train_X = train[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
train_Y = train.Species
test_X = test[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
test_Y = test.Species

In [44]:
train_X.head(2)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
22,4.6,3.6,1.0,0.2
15,5.7,4.4,1.5,0.4


In [45]:
test_X.head(2)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
73,6.1,2.8,4.7,1.2
18,5.7,3.8,1.7,0.3


In [46]:
train_Y.head(2)

22    Iris-setosa
15    Iris-setosa
Name: Species, dtype: object

In [47]:
test_Y.head(2)

73    Iris-versicolor
18        Iris-setosa
Name: Species, dtype: object

In [49]:
#Fit the model
# classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
# classifier.fit(X_train, Y_train)
# classifier.print_tree()

from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
model.fit(train_X, train_Y)
prediction=model.predict(test_X)
print('The accuracy of the decision tree :', metrics.accuracy_score(prediction, test_Y))


ValueError: ignored